In [1]:
import openai
import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')

# Example

## Story Generation

In [2]:
import openai

prompt_system = "You are a helpful assistant whose goal is to help write stories."

prompt = """Continue the following story. Write no more than 40 words.
Once upon a time, in a world where animal could speak, a courageous mouse named Benjamin decided to"""

client = openai.Client(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"system", "content":prompt_system},
        {"role":"user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

embark on a quest to find the mythical cheese of legends. He knew it was a dangerous journey, but his determination never wavered.


## Product Description

In [3]:
import openai

prompt_system = """You are a helpful assistant whose goal is to help write product descriptions."""
prompt = """Write a captivating product description for a luxurious, handcrafted, limited-edition fountain pen made from rosewood and gold.
Write no more than 50 words"""

client = openai.Client(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"system", "content":prompt_system},
        {"role":"user", "content":prompt}
    ]
)

print(response.choices[0].message.content)

Indulge in the exquisite craftsmanship of our limited-edition fountain pen, meticulously handcrafted from rich rosewood and decadent gold. Elevate your writing experience with this luxurious instrument that combines elegance and sophistication, perfect for discerning connoisseurs of fine stationery.


# Prompting Techniques

## 1. Zero-Shot Prompting

In [4]:
import openai

prompt_system = "You are a helpful assistant whose goal is to write short poems."
prompt = """Write a short poem about {topic}."""

client = openai.Client(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"system", "content":prompt_system},
        {"role":"user", "content":prompt.format(topic="summer")}
    ]
)
print(response.choices[0].message.content)

Golden sun and skies so clear,
Summer's warmth is drawing near.
Birds are singing, flowers bloom,
Nature's beauty fills the room.

Lazy days and endless nights,
Joyful laughter and fun delights.
Beaches, picnics, and ice cream treats,
Warm embraces and bare feet.

Summer's magic in the air,
Moments cherished, memories rare.
Let the sunshine kiss your skin,
Embrace the season from within.


## 2. In-Context learning and Few-Shot Prompting

In [5]:
import openai

prompt_system = "You are a helpful assistant whose goal is to write short porms."
prompt = """Write a short poem about {topic}."""

examples = {
    "nature": """Birdsong fills the air,\nMountains high and valleys deep,\nNature's music sweet.""",
    "winter": """Snow blankets the ground,\nSilence is the only sound,\nWinter's beauty found."""
}

client = openai.Client(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"system", "content":prompt_system},
        {"role":"user", "content":prompt.format(topic="nature")},
        {"role":"assistant", "content":examples["nature"]},
        {"role":"user", "content":prompt.format(topic="winter")},
        {"role":"assistant", "content":examples["winter"]},
        {"role":"user", "content":prompt.format(topic="summer")}
    ]
)

print(response.choices[0].message.content)

Golden sunbeams dance,
Laughter in the warm breeze,
Summer's joy enhance.


### Few-Shot Prompting Example

In [7]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

examples = [
    {"color":"red", "emotion":"passion"},
    {"color":"blue", "emotion":"serenity"},
    {"color":"green", "emotion":"tranquility"}
]

example_formatter_template = """
Color:{color}
Emotion: {emotion}\n
"""

example_prompt = PromptTemplate(
    input_variables = ["color", "emotion"],
    template=example_formatter_template
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix = """Here are some examples of colors and the emotions associated with them:\n\n""",
    suffix = """\n\nNow, given a new color, identify the emotion associated with it:\n\nColor:{input}\nEmotion:""",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="purple")

# create the LLM for the prompt
chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))

# run the LLMChain to get the AI-generated emotion associated with the input
# color
response = chain.run({})
print("Color:purple")
print("Emotion:", response)

c:\Users\woojin\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Color:purple
Emotion: royalty


## 3. Role Prompting

In [8]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

# initialize LLM
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0)

template = """ 
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""

prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template
)

# create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=prompt)

# inpout data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"}

# run the LLMChain to get the AI-generated song title
response = chain.run(input_data)

print("These: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response)

These: interstellar travel
Year: 3030
AI-generated song title: "Galactic Odyssey: Journey Through the Stars in 3030"


## 4. Chain Prompting

In [12]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

# initalize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

# prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer:"""
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer:"""
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# run the LLMChain for the first prmompt with an empty dictionary
response_question = chain_question.run({})

# extract the scientist's name from the response
scientist = response_question.strip()

# create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# input data for the second prompt
input_data = {"scientist": scientist}

# run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)


Scientist: Albert Einstein
Fact: Albert Einstein's theory of general relativity is a theory of gravitation that describes the force of gravity as a curvature of spacetime caused by mass and energy. According to this theory, massive objects like planets and stars warp the fabric of spacetime, causing other objects to move along curved paths. General relativity also predicts phenomena such as gravitational time dilation, gravitational waves, and the bending of light around massive objects.


# Bad Prompt Practices

In [13]:
template = "Tell me something about {topic}."
prompt = PromptTemplate(
    input_variables=['topic'],
    template=template
)
prompt.format(topic='dogs')

'Tell me something about dogs.'

In [14]:
# prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer:"""
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# promtp 2
template_fact = """Tell me something interesting about {scientist}.
Answer:"""
prompt_fact = PromptTemplate(input_variables=['scientist'], template=template_fact)

# create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# run the LLMChain for the first prompt with an empty dictionay
response_question = chain_question.run({})

# extract the scientist's nam from the response
scientist = response_question.strip()

# cretate the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# input data for the second prompt
input_data = {"scientist": scientist}

# run the LLMChain for the second prompt
response_dact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Albert Einstein
Fact: Albert Einstein's theory of general relativity is a theory of gravitation that describes the force of gravity as a curvature of spacetime caused by mass and energy. According to this theory, massive objects like planets and stars warp the fabric of spacetime, causing other objects to move along curved paths. General relativity also predicts phenomena such as gravitational time dilation, gravitational waves, and the bending of light around massive objects.


In [17]:
# prompt 1
template_prompt = """What are some musical genres?
Answer:"""
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# prompt 2
template_fact = """Tell me something about {genre1}, {genre2}, and {genre3} without giving without any specific details
Answer:"""
prompt_fact = PromptTemplate(input_variables=['genre1', 'genre2', 'genre3'], template=template_fact)

# create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# assign three hardcoded genre
genre1, genre2, genre3 = "jazz", "pop", "rock"

# create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# input data for the second prompt
input_data = {"genre1": genre1, "genre2": genre2, "genre3":genre3}

# run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Genres:", genre1, genre2, genre3)
print("Fact:", response_fact)

Genres: jazz pop rock
Fact: Jazz is known for its improvisational nature, pop music is characterized by catchy melodies and hooks, and rock music is often associated with electric guitars and high energy performances.


# Tips for Effective Prompt Engineering

In [21]:
from langchain import FewShotPromptTemplate, PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

# initialize LLM
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=OPENAI_API_KEY)
examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": """Finding balance in life and learning to enjoy the small moments."""
    }, {
        "query": "How can I become more productive?",
        "answer": """Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."""
    }
]
example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables = ["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI life coach.
The assistant provides insightful and practical adive to the users' questions. Here are some examples:"""

suffix = """
User: {query},
AI:
"""

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['query'],
    example_separator="\n\n"
)

# create the LLMChain for the few-shot prompt temaplte
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# define the user query
user_query = "What are some tips for improving communication skills?"

# run the LLMChain for the user query
response = chain.run({"query": user_query})

print("User Query:", user_query)
print("All Response:", response)

User Query: What are some tips for improving communication skills?
All Response: Practice active listening, be mindful of your body language, and work on expressing yourself clearly and confidently.
